In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['TORCH_HOME'] = "/disks/sdb/torch_home"

import logging
logging.getLogger().setLevel(logging.WARNING)

from awesome_glue.task import *
from awesome_glue.config import Config

Loading faiss with AVX2 support.
I0309 07:44:05.957231 139905671493440 file_utils.py:38] PyTorch version 1.4.0 available.
I0309 07:44:07.491601 139905671493440 modeling.py:230] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
I0309 07:44:07.590372 139905671493440 modeling_bert.py:244] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
[nltk_data] Downloading package wordnet to /home/zhouyi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/zhouyi/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [26]:
from allennlp.data.fields import TextField
from allennlp.data.tokenizers import SpacyTokenizer, CharacterTokenizer
from allennlp.data.token_indexers import SingleIdTokenIndexer, TokenCharactersIndexer
tf = TextField(SpacyTokenizer().tokenize("hello world, hello everyone"), 
               {"spp": SingleIdTokenIndexer(), 
                "chr": TokenCharactersIndexer()})

/opt/anaconda3/lib/python3.7/site-packages/allennlp/data/token_indexers/token_characters_indexer.py:59: UserWarning: You are using the default value (0) of `min_padding_length`, which can cause some subtle bugs (more info see https://github.com/allenai/allennlp/issues/1954). Strongly recommend to set a value, usually the maximum size of the convolutional layer size when using CnnEncoder.
  UserWarning,


In [60]:
from typing import Iterator, List, Dict
import torch
import torch.optim as optim
import numpy as np
from allennlp.data import Instance
from allennlp.data.fields import TextField, SequenceLabelField
from allennlp.data.dataset_readers import DatasetReader
from allennlp.common.file_utils import cached_path
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer, TokenCharactersIndexer, PretrainedTransformerIndexer
from allennlp.data.tokenizers import Token
from allennlp.data.vocabulary import Vocabulary
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits
from allennlp.training.metrics import CategoricalAccuracy
from allennlp.training.trainer import Trainer
from allennlp.predictors import SentenceTaggerPredictor

torch.manual_seed(1)
class PosDatasetReader(DatasetReader):
    """
    DatasetReader for PoS tagging data, one sentence per line, like

        The###DET dog###NN ate###V the###DET apple###NN
    """
    def __init__(self, token_indexers: Dict[str, TokenIndexer] = None) -> None:
        super().__init__(lazy=False)
        self.token_indexers = token_indexers or {"tokens": SingleIdTokenIndexer()}
    def text_to_instance(self, tokens: List[Token], tags: List[str] = None) -> Instance:
        sentence_field = TextField(tokens, self.token_indexers)
        fields = {"sentence": sentence_field}

        if tags:
            label_field = SequenceLabelField(labels=tags, sequence_field=sentence_field)
            fields["labels"] = label_field

        return Instance(fields)
    def _read(self, file_path: str) -> Iterator[Instance]:
        with open(file_path) as f:
            for line in f:
                pairs = line.strip().split()
                sentence, tags = zip(*(pair.split("###") for pair in pairs))
                yield self.text_to_instance([Token(word) for word in sentence], tags)
class LstmTagger(Model):
    def __init__(self,
                 word_embeddings: TextFieldEmbedder,
                 encoder: Seq2SeqEncoder,
                 vocab: Vocabulary) -> None:
        super().__init__(vocab)
        self.word_embeddings = word_embeddings
        self.encoder = encoder
        self.hidden2tag = torch.nn.Linear(in_features=encoder.get_output_dim(),
                                          out_features=vocab.get_vocab_size('labels'))
        self.accuracy = CategoricalAccuracy()
    def forward(self,
                sentence: Dict[str, torch.Tensor],
                labels: torch.Tensor = None) -> Dict[str, torch.Tensor]:
        mask = get_text_field_mask(sentence)
        embeddings = self.word_embeddings(sentence)
        encoder_out = self.encoder(embeddings, mask)
        tag_logits = self.hidden2tag(encoder_out)
        output = {"tag_logits": tag_logits}
        if labels is not None:
            self.accuracy(tag_logits, labels, mask)
            output["loss"] = sequence_cross_entropy_with_logits(tag_logits, labels, mask)

        return output
    def get_metrics(self, reset: bool = False) -> Dict[str, float]:
        return {"accuracy": self.accuracy.get_metric(reset)}
    
# "tfm": PretrainedTransformerIndexer("bert-base-uncased")
reader = PosDatasetReader({"sps": SingleIdTokenIndexer(), "chr": TokenCharactersIndexer(), })
train_dataset = reader.read(cached_path(
    'https://raw.githubusercontent.com/allenai/allennlp'
    '/master/tutorials/tagger/training.txt'))
validation_dataset = reader.read(cached_path(
    'https://raw.githubusercontent.com/allenai/allennlp'
    '/master/tutorials/tagger/validation.txt'))
vocab = Vocabulary.from_instances(train_dataset + validation_dataset)
train_dataset.index_with(vocab)
validation_dataset.index_with(vocab)

I0309 09:58:03.650615 139905671493440 vocabulary.py:258] Fitting token dictionary from dataset.


In [68]:
tf = train_dataset[1]['sentence']
print(tf.tokens)
tf.get_padding_lengths()

[Everybody, read, that, book]


{'sps___tokens': 4,
 'chr___token_characters': 4,
 'chr___num_token_characters': 9}

In [73]:
tf1 = tf.as_tensor(tf.get_padding_lengths())
train_dataset[1].as_tensor_dict()

{'sentence': {'sps': {'tokens': tensor([ 7,  8,  9, 10])},
  'chr': {'token_characters': tensor([[15, 16,  2,  9, 10, 11,  3,  7, 10],
           [ 9,  2,  4,  7,  0,  0,  0,  0,  0],
           [ 5,  6,  4,  5,  0,  0,  0,  0,  0],
           [11,  3,  3, 17,  0,  0,  0,  0,  0]])}},
 'labels': tensor([0, 2, 1, 0])}

In [2]:
toy_data = load_data('TOY', 'spacy')

cache for TOY-spacy.data exists
*** load TOY-spacy.data from cache cost 3.53 seconds


In [9]:
dev_data =  toy_data['data'][1]
dev_data.vocab

In [52]:
val_data = toy_data['data'][1]
from allennlp.data.dataset_readers.dataset_reader import AllennlpDataset
ds = AllennlpDataset(val_data, toy_data['vocab'])

In [72]:
ds.instances[5].fields['sent'].index(ds.vocab)

TypeError: tokens_to_indices() takes 3 positional arguments but 4 were given

In [32]:
config = Config()
task = Task(config)
task.from_pretrained()
config

cache for SST-spacy.data exists
*** load SST-spacy.data from cache cost 3.36 seconds
cache for SST-glove.vec exists
*** load SST-glove.vec from cache cost 0.0123 seconds


Basic Args:
	--task_id=SST
	--finetunable=True
	--arch=lstm
	--pretrain=glove
	--_model_name=
	--mode=transfer
	--adv_data=nogit/SST-lstm.hotflip.adv.tsv
	--transform=embed_aug
	--randomness=False
	--aug_data=
	--attack_method=hotflip
	--attack_vectors=counter
	--attack_data_split=dev
	--attack_size=400
	--attack_gen_aug=False
	--attack_gen_adv=True
	--alchemist=False
	--seed=2
Deduced Args:
	--model_name=SST-lstm
	--tokenizer=spacy

In [41]:
config.attack_size = 50
config.attack_method = "pwws"
task.attack()

cache for SST-counter.vec exists
*** load SST-counter.vec from cache cost 0.0135 seconds



  0%|          | 0/50 [00:00<?, ?it/s]

Sanity Test



  8%|▊         | 4/50 [00:01<00:16,  2.84it/s]

[raw] the acting , costumes , music , cinematography and sound are all astounding given the production 's austere locales .
	 ['  0.0303', '  0.9697']
[adv] the acting , costumes , music , cinematography nor sound are all astounding given the production 's austere locales .
	 ['  0.8179', '  0.1821']
[changed] 1

Avg.change# 1.0 Avg.change% 5.0
Aggregated metric: Accu before: 100.00%, after: 66.67%, Flip ratio 33.33%


...

[raw] a synthesis of cliches and absurdities that seems positively decadent in its cinematic flash and emptiness .
	 ['  0.9636', '  0.0364']
[adv] a synthesis of sociological and absurdities that seems positively decadent in its cinematic wink and emptiness .
	 ['  0.4770', '  0.5230']
[changed] 2

Avg.change# 1.68 Avg.change% 8.35
Aggregated metric: Accu before: 86.96%, after: 45.65%, Flip ratio 47.50%



 94%|█████████▍| 47/50 [00:22<00:01,  2.24it/s]

[raw] a subtle and well - crafted ( for the most part ) chiller .
	 ['  0.0015', '  0.9985']
[adv] a subtle nor well - crafted ( for the most part ) chiller .
	 ['  0.7256', '  0.2744']
[changed] 1

Avg.change# 1.65 Avg.change% 8.29
Aggregated metric: Accu before: 87.23%, after: 44.68%, Flip ratio 48.78%



100%|██████████| 50/50 [00:24<00:00,  2.04it/s]

[raw] it feels like an after - school special gussied up with some fancy special effects , and watching its rote plot points connect is about as exciting as gazing at an egg timer for 93 minutes .
	 ['  0.9829', '  0.0171']
[adv] it discoveries like an after - school unique gussied up with some fancy special effects , and watching its rouge plot points connect is about as exciting as gazing at an egg timer for 93 record .
	 ['  0.2827', '  0.7173']
[changed] 4

Avg.change# 1.76 Avg.change% 8.41
Aggregated metric: Accu before: 88.00%, after: 46.00%, Flip ratio 47.73%
Avg.change# 1.76 Avg.change% 8.41
Accu before: 88.00%, after: 46.00%, Flip ratio 47.73%


In [18]:
scores = {}
stops = stopwords.words("english")
all_words = list(task.vocab.get_index_to_token_vocabulary().values())[2:]
for word in all_words:
    if "'" in word:
        continue
    scores[word] = round(task.predictor.predict(word)['probs'][0], 2)

In [130]:
# from awesome_glue.utils import FreqUtil
# FreqUtil.split_by_frequency(task.vocab, 19, 20)
task.vocab._retained_counter['tokens']['Microsoft']

6375

TypeError: 'Vocabulary' object is not subscriptable

In [75]:
total = 0
for i in range(30):
    sent = allenutil.as_sentence(task.dev_data[i])
    before = task.predictor.predict(sent)['probs'][0]
    if before > 0.5:
        sent = "above above " + sent
    else:
        sent = "too too " + sent
    after = task.predictor.predict(sent)['probs'][0]
    if (before - 0.5) * (after - 0.5) < 0:
        total += 1
print(total)

11


In [27]:
sorted(scores.items(), key=lambda x:x[1], reverse=False)[:5000][-1]

('voices', 0.13)

In [145]:
task.vocab.get_token_index('d')

12732